# Reddit ETL Pipeline - Database Query Notebook

This notebook allows you to query the PostgreSQL database used by the Reddit ETL pipeline.

## Database Connection Details
- **Configuration**: Loaded from `../config/config.conf`
- **Security**: No hardcoded credentials in the notebook
- **Environment**: Uses the same config as the ETL pipeline


In [ ]:
# Import required libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import configparser
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path to access config
sys.path.append('..')

print("Libraries imported successfully!")


In [ ]:
# Load database configuration using the project's config loader
from utils.config_loader import load_config

# Load configuration
try:
    config = load_config('../config/config.conf')
    DB_CONFIG = config.get_database_config()
    connection_string = config.get_connection_string()
    
    print("✅ Database configuration loaded from config file!")
    print(f"📊 Connected to: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
    print("🔒 Using secure configuration loading (no hardcoded credentials)")
    
except Exception as e:
    print(f"❌ Error loading database configuration: {e}")
    print("Make sure you're running this notebook from the notebooks/ directory")
    print("and that ../config/config.conf exists")
    print("Also ensure the utils/config_loader.py file is available")


In [ ]:
# Test database connection
try:
    # Test with psycopg2
    conn = psycopg2.connect(**DB_CONFIG)
    print("✅ Database connection successful!")
    
    # Get database info
    cursor = conn.cursor()
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()[0]
    print(f"📊 Database Version: {db_version}")
    
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"❌ Database connection failed: {e}")


In [ ]:
# Create SQLAlchemy engine for pandas integration
try:
    engine = create_engine(connection_string)
    print("✅ SQLAlchemy engine created successfully!")
except Exception as e:
    print(f"❌ SQLAlchemy engine creation failed: {e}")


## Configuration Overview

The notebook uses the same configuration file as the ETL pipeline. Here's what's available:


In [ ]:
# Display available configuration sections
try:
    print("🔧 Available Configuration Sections:")
    print("=" * 50)
    
    # Database config
    db_config = config.get_database_config()
    print(f"📊 Database: {db_config['host']}:{db_config['port']}/{db_config['database']}")
    
    # AWS config
    aws_config = config.get_aws_config()
    print(f"☁️  AWS Region: {aws_config['region']}")
    print(f"🪣 S3 Bucket: {aws_config['bucket_name']}")
    
    # Reddit config
    reddit_config = config.get_reddit_config()
    print(f"🔗 Reddit Client ID: {reddit_config['client_id'][:8]}...")
    
    # File paths
    file_paths = config.get_file_paths()
    print(f"📁 Output Path: {file_paths['output_path']}")
    
    # ETL settings
    etl_settings = config.get_etl_settings()
    print(f"⚙️  Batch Size: {etl_settings['batch_size']}")
    print(f"📝 Log Level: {etl_settings['log_level']}")
    
    print("\n✅ All configuration loaded successfully!")
    
except Exception as e:
    print(f"❌ Error displaying configuration: {e}")


## Database Schema Exploration

Let's explore what tables exist in the database.


In [ ]:
# List all tables in the database
query_tables = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
ORDER BY table_name;
"""

try:
    tables_df = pd.read_sql(query_tables, engine)
    print("📋 Available Tables:")
    print(tables_df)
    
    if len(tables_df) == 0:
        print("\n⚠️  No tables found in the public schema.")
        print("This might mean the ETL pipeline hasn't run yet or data hasn't been loaded.")
    
except Exception as e:
    print(f"❌ Error listing tables: {e}")


## Sample Queries

Here are some sample queries you can run. Modify them as needed for your analysis.


In [ ]:
# Query 1: Check if reddit_posts table exists and get sample data
query_reddit_posts = """
SELECT * 
FROM reddit_posts 
LIMIT 5;
"""

try:
    reddit_posts_df = pd.read_sql(query_reddit_posts, engine)
    print("📊 Reddit Posts Sample Data:")
    print(reddit_posts_df)
    print(f"\n📈 Total rows: {len(reddit_posts_df)}")
except Exception as e:
    print(f"❌ Error querying reddit_posts: {e}")
    print("This table might not exist yet. Run the ETL pipeline first.")


In [ ]:
# Query 2: Check sentiment analysis results
query_sentiment = """
SELECT * 
FROM sentiment_analysis 
LIMIT 5;
"""

try:
    sentiment_df = pd.read_sql(query_sentiment, engine)
    print("📊 Sentiment Analysis Sample Data:")
    print(sentiment_df)
    print(f"\n📈 Total rows: {len(sentiment_df)}")
except Exception as e:
    print(f"❌ Error querying sentiment_analysis: {e}")
    print("This table might not exist yet. Run the ETL pipeline first.")


## Custom Queries

Use the cell below to run your own custom queries.


In [ ]:
# Custom Query - Modify this query as needed
custom_query = """
-- Example: Get all tables and their row counts
SELECT 
    schemaname,
    tablename,
    n_tup_ins as inserts,
    n_tup_upd as updates,
    n_tup_del as deletes
FROM pg_stat_user_tables
ORDER BY tablename;
"""

try:
    custom_df = pd.read_sql(custom_query, engine)
    print("📊 Custom Query Results:")
    print(custom_df)
except Exception as e:
    print(f"❌ Error running custom query: {e}")
